In [25]:
import os, sys
from scapy.all import *
load_contrib('bgp')

In [2]:
base = "/home/greg/BGP_Chain/Seed_scalable_complete/bgp_smart_contracts/pcaps/"
# base = "/Users/gcusack/Downloads/"
pcap_path = base + "multiple-updates.pcap"
# pcap_path = base + "update.pcap"

In [4]:
def get_bgp_update_packet():
    for pkt in PcapReader(pcap_path):
        try:
            if (str(pkt.summary()).find('BGPHeader') > 0) and (pkt[BGPHeader].type == 2):
                return pkt
                print(pkt[TCP].show())
    #             print(packet[TCP].payload())
        except Exception as e:
            print(repr(e))

In [5]:
pkt = get_bgp_update_packet()

In [6]:
pkt.show()

###[ cooked linux ]### 
  pkttype   = unicast
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nh\x00\x0c'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 641
     id        = 4998
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x4db2
     src       = 10.104.0.12
     dst       = 10.104.0.164
     \options   \
###[ TCP ]### 
        sport     = bgp
        dport     = 59473
        seq       = 2063855751
        ack       = 61297140
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 510
        chksum    = 0x17f3
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3262136288, 1326917326))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 152
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withdrawn_

In [7]:
type(pkt)
dir(pkt)

['_PickleType',
 '__all_slots__',
 '__annotations__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__iterlen__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__orig_bases__',
 '__parameters__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_answered',
 '_do_summary',
 '_is_protocol',
 '_name',
 '_overload_fields',
 '_pkt',
 '_resolve_alias',
 '_show_or_dump',
 '_superdir',
 'add_payload',
 'add_underlayer',
 'aliastypes',
 'answers',
 'build',
 'build_do

In [8]:
bgp_update_1 = pkt.getlayer(scapy.contrib.bgp.BGPUpdate, 1)

In [9]:
bgp_update_1.show()

###[ UPDATE ]### 
  withdrawn_routes_len= 0
  \withdrawn_routes\
  path_attr_len= 83
  \path_attr \
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = ORIGIN
   |  attr_len  = 1
   |  \attribute \
   |   |###[ ORIGIN ]### 
   |   |  origin    = IGP
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = AS_PATH
   |  attr_len  = 18
   |  \attribute \
   |   |###[ AS_PATH (RFC 4893) ]### 
   |   |  \segments  \
   |   |   |###[ ASPathSegment ]### 
   |   |   |  segment_type= AS_SEQUENCE
   |   |   |  segment_length= 4
   |   |   |  segment_value= [12, 3, 11, 11872]
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = NEXT_HOP
   |  attr_len  = 4
   |  \attribute \
   |   |###[ NEXT_HOP ]### 
   |   |  next_hop  = 10.104.0.12
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive+Optional
   |  type_code = 32
   |  attr_len  = 48
   |  \attribute \
   |   |###[ Raw ]### 
   |   |  load      = '\x00\x00\x00\x03\x00\x00\x00\x01

In [11]:
# bgp_update_1.summary()
len(bgp_update_1.path_attr)
bgp_update_1.path_attr
for i in bgp_update_1.path_attr:
    if i.type_code == 0x02: # 0x02 == AS_PATH
        print(i.attribute.show())
        print(type(i.attribute))
        print(i.attribute.segments[0].segment_value)

###[ AS_PATH (RFC 4893) ]### 
  \segments  \
   |###[ ASPathSegment ]### 
   |  segment_type= AS_SEQUENCE
   |  segment_length= 4
   |  segment_value= [12, 3, 11, 11872]

None
<class 'scapy.contrib.bgp.BGPPAAS4BytesPath'>
[12, 3, 11, 11872]


In [21]:
# bgp_update_1.summary()
next_hop = None
len(bgp_update_1.path_attr)
bgp_update_1.path_attr
for i in bgp_update_1.path_attr:
    if i.type_code == 0x03: # 0x02 == AS_PATH
        print(i.attribute.show())
        print(i.attribute.next_hop)
        next_hop = i.attribute.next_hop
#         print(type(i.attribute))
#         print(i.attribute.segments[0].segment_value)

###[ NEXT_HOP ]### 
  next_hop  = 10.104.0.12

None
10.104.0.12


In [26]:
print(next_hop)
ret = subprocess.check_output('birdc "sho route where bgp_path ~[= * =]" all | grep ' + next_hop + ' | grep ix', shell=True).decode('utf-8')
ret.split("ix", 1)[1].strip()

10.104.0.12


/bin/sh: 1: birdc: not found


CalledProcessError: Command 'birdc "sho route where bgp_path ~[= * =]" all | grep 10.100.0.151 | grep ix' returned non-zero exit status 1.

In [13]:
bgp_update_1.path_attr[1]

<BGPPathAttr  type_flags=Transitive type_code=AS_PATH attr_len=18 attribute=<BGPPAAS4BytesPath  segments=[<ASPathSegment  segment_type=AS_SEQUENCE segment_length=4 segment_value=[12, 3, 11, 11872] |>] |> |>

In [12]:
# pkt.layers()
# pkt.getlayer(scapy.contrib.bgp.BGPHeader).show()
payload_of_interest = None
most_recent_bgp_header = None
for count, payload in enumerate(pkt.iterpayloads()):
    print(count)
    if isinstance(payload,  scapy.contrib.bgp.BGPHeader):
        most_recent_bgp_header = payload
    elif isinstance(payload, scapy.contrib.bgp.BGPUpdate):
        print(type(payload))
        nlri_asn_check_v2(most_recent_bgp_header, payload)
#         payload_of_interest = payload
#         break
    print("##################################")

0
##################################
1
##################################
2
##################################
3
##################################
4
<class 'scapy.contrib.bgp.BGPUpdate'>


NameError: name 'nlri_asn_check_v2' is not defined

In [ ]:
def nlri_asn_check_v2(most_recent_bgp_header, payload):
#     print(payload.show())
#     eval(payload).withdrawn_route_klen
    if payload.withdrawn_routes_len == 0 and payload.path_attr_len != 0:
#         print(payload.path_attr[1].attribute.segments[]
#         print(payload.show())
        if len(payload.path_attr[1].attribute.segments[-1].segment_value) > 1:
           asn = payload.path_attr[1].attribute.segments[-1].segment_value[-1]
        else:
           print('reverting to segment length')
           asn = payload.path_attr[1].attribute.segments[-1].segment_length
        
        for num, item in enumerate(payload.nlri):
            segment = [asn, item.prefix.split('/')[0], item.prefix.split('/')[1]]
            print(segment)


In [ ]:
pkt.show()

In [ ]:
pkt.summary()

In [ ]:
if (str(pkt.summary()).find('BGPHeader') > 0) and (pkt[BGPHeader].type == 2) :
        orig_len=pkt[IP].len

        cmd_lst_2=scapy_command_builder(pkt,"pkt")
#         print(cmd_lst_2)
#         for i in cmd_lst_2:
#             print(type(i))

        bgp_update_checker(cmd_lst_2,pkt)

#         pkt_final=packet_prepper(pkt)
#         pkt.show2()
#         print ('Original packet length: '+str(orig_len) + ' New packet length: ' +str(pkt_final[IP].len))

In [ ]:
def pkt_assembler(header,payload):
    print('assembling packet')
    pkt_assem='header[0]/header[1]/header[2]'
    for num in range(len(payload)):
        pkt_assem+='/payload['+str(num)+']'
    final_pkt=eval(pkt_assem)
    return final_pkt

In [ ]:

def scapy_command_builder(pkt,tmp_str):
    print('building commands')
    x=pkt.summary().split(' / ')
#     print(x)
    x[1]='TCP'
    x=expander(x,tmp_str)
#     print ('commands are')
#     print (x)
    return x

In [ ]:
def expander(lst,tmp_str):
     print('expanding commands')
     w=''
     s=[]
     for item in lst[:]:
         w+='[ ' + str(item) + ' ]'
         s.append(tmp_str+'['+w[1:-1]+']')
     return s

In [ ]:
def packet_header(header_cmd,pkt2):
     print('packet header')
     del header_cmd[3:]
     header=[]
     for n in range(0,3):
         eval(header_cmd[-1-n]).remove_payload()
         header.append(eval(header_cmd[-1-n]))
     header.reverse()
     return header

In [ ]:
def packet_prepper(pkt):
   print('packet prepper')
   pkt[IP].len
   #pkt.show()
   del pkt[IP].chksum
   del pkt[TCP].chksum
   return pkt

In [ ]:
def bgp_update_checker(cmd_lst,pkt):
     print('checking packet')
     del cmd_lst[:3]
     #print (cmd_lst)
     print(" Break ============")
     pkt_placeholder=[]
     for n in range(len(cmd_lst)):
         print('testing ...\n' + str(cmd_lst[(-1-n)]))
         if  str(cmd_lst[(-1-n)]).endswith('[ BGPUpdate ]'):
            print(type(cmd_lst[(-1-n)]))
#             print(str(cmd_lst[(-1-n)]))
#              nlri_asn_check(str(cmd_lst[(-1-n)]), pkt)

         else:
            pass

In [ ]:
def nlri_asn_check(cmd, pkt):
    print('conducting NLRI check....')
    print('checking: '+cmd)
    #print(pkt.show())
    print('trying ASN Identification...')
    print(str(eval(cmd).withdrawn_routes_len))   
    if ((eval(cmd).withdrawn_routes_len == 0) and (eval(cmd).path_attr_len != 0)):
        print(pkt.show())
        print(str(eval(cmd).path_attr[1].attribute.segments[-1].segment_value))
        print('entering if statement')
        if len(eval(cmd).path_attr[1].attribute.segments[-1].segment_value) > 1:
           asn=eval(cmd).path_attr[1].attribute.segments[-1].segment_value[-1]
        else:
           print('reverting to segment length')
           asn=eval(cmd).path_attr[1].attribute.segments[-1].segment_length
        #nlri_lst=copy.deepcopy(eval(cmd).nlri) 
        print('ASN is: '+str(asn) + ' Checking prefixes:....')
        print(eval(cmd).nlri)
        num=0
        for item in eval(cmd).nlri:  
            segment=[asn, eval(cmd).nlri[num].prefix.split('/')[0], eval(cmd).nlri[num].pref]
            check=bgpchain_validate(segment, tx_sender)
            #print ('Placeholder evaluation check for: ' + str(adv_segment[0]) + ' ' + str(a>
            #check='Authorized'
            print
            if check =='Authorized':
               num+=1
               pass
            else:
               print('deleting prefix...')
               eval(cmd).nlri[num].prefix = '0.0.0.0/0'
               print (eval(cmd).nlri)
               num+=1
              # if len(eval(cmd).nlri) == 1:
               #   eval(cmd).nlri[0].prefix ='0.0.0.0/0'
                #  print(eval(cmd).nlri)

        #eval(cmd).nlri=nlri_lst
        #return eval(cmd)
    else:
        print('Segment was a route withdrawl')
        return
        #return eval(cmd)